In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
df = pd.read_csv("luka_dataset.csv",index_col=0)
df.head()

,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,BLK,TOV,PF,PTS,GmSc,"""+/-""",Win,Winby,Minutes,age_y
0,1,5,16,0.313,0,5,0.000,0,2,0.000,...,0,4,3,10,0.8,-18,0,-21,31.783333,19.632877
1,2,8,16,0.500,4,9,0.444,6,9,0.667,...,1,6,3,26,16.6,6,1,4,36.233333,19.641096
2,3,7,14,0.500,3,8,0.375,2,2,1.000,...,0,3,3,19,13.9,5,1,6,35.800000,19.646575
3,4,7,18,0.389,2,9,0.222,5,8,0.625,...,1,3,2,21,12.0,-12,0,-7,34.183333,19.652055
4,5,7,14,0.500,4,6,0.667,4,4,1.000,...,0,3,1,22,16.3,0,0,-9,35.150000,19.657534


In [7]:
df = df.astype('float32')

In [8]:
y = df['Win']
X = df.drop(columns=['Win','Winby'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [10]:
def my_DL(epochs=6,batchsize=512):
    model = Sequential()
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [11]:
def train_hyper_tune(X,y):
    # create the pre-processing component
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    # define classifiers
    ## Classifier 1: Logistic Regression
    clf_LR = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga')
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)
    ## Classifier 3: Deep Learning Binary Classifier
    clf_DL = KerasClassifier(build_fn=my_DL)
    
    # define pipeline for three classifiers
    ## clf_LR
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('lr_model',clf_LR)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
    ## clf_DL
    pipe3 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dl_model',clf_DL)])
    
    # create hyperparameter space of the three models
    ## clf_LR
    param_grid1 = {
        'lr_model__C' : [1e-1,1,10],
        'lr_model__l1_ratio' : [0,0.5,1]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,5]
    }
    ## clf_DL
    param_grid3 = {
        'dl_model__epochs' : [6,12,18,24],
        'dl_model__batchsize' : [256,512]
    }
    
    # set GridSearch via 5-fold cross-validation
    ## clf_LR
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
    ## clf_DL
    grid3 = GridSearchCV(pipe3, cv=5, param_grid=param_grid3)
    
    # run the hyperparameter tunning process
    grid1.fit(X,y)
    grid2.fit(X,y)
    grid3.fit(X,y)
    
    # return results of the tunning process
    return grid1,grid2,grid3,pipe1,pipe2,pipe3

In [12]:
my_grid1,my_grid2,my_grid3,my_pipe1,my_pipe2,my_pipe3 = train_hyper_tune(X_train, y_train)

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6724 - accuracy: 0.5227
Epoch 2/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6397 - accuracy: 0.5682
Epoch 3/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6187 - accuracy: 0.6364
Epoch 4/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6028 - accuracy: 0.7159
Epoch 5/6
3/3 [==============================] - 0s 3ms/step - loss: 0.5888 - accuracy: 0.7273
Epoch 6/6
1/1 [==============================] - 0s 827us/step - loss: 0.6375 - accuracy: 0.7391
Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6910 - accuracy: 0.5281
Epoch 2/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6478 - accuracy: 0.6180
Epoch 3/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6240 - accuracy: 0.6854
Epoch 4/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6048 - accuracy: 0.6966
Epoch 5/6
3/3 [==============================] -

3/3 [==============================] - 0s 1ms/step - loss: 0.8018 - accuracy: 0.4831
Epoch 3/12
3/3 [==============================] - 0s 2ms/step - loss: 0.7653 - accuracy: 0.5056
Epoch 4/12
3/3 [==============================] - 0s 1ms/step - loss: 0.7381 - accuracy: 0.5281
Epoch 5/12
3/3 [==============================] - 0s 1ms/step - loss: 0.7134 - accuracy: 0.5281
Epoch 6/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6944 - accuracy: 0.5730
Epoch 7/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6786 - accuracy: 0.5730
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6627 - accuracy: 0.5730
Epoch 9/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6474 - accuracy: 0.5843
Epoch 10/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6346 - accuracy: 0.6067
Epoch 11/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.6180
Epoch 12/12
1/1 [==============================] 

1/1 [==============================] - 0s 739us/step - loss: 0.6113 - accuracy: 0.6087
Epoch 1/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6811 - accuracy: 0.6067
Epoch 2/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6473 - accuracy: 0.6404
Epoch 3/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6252 - accuracy: 0.6854
Epoch 4/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6078 - accuracy: 0.7303
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5926 - accuracy: 0.7416
Epoch 6/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5778 - accuracy: 0.7978
Epoch 7/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5659 - accuracy: 0.8090
Epoch 8/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5534 - accuracy: 0.8202
Epoch 9/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5412 - accuracy: 0.8202
Epoch 10/18
3/3 [==============================] 

3/3 [==============================] - 0s 1ms/step - loss: 0.6852 - accuracy: 0.5281
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6679 - accuracy: 0.5393
Epoch 6/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6518 - accuracy: 0.5618
Epoch 7/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6381 - accuracy: 0.5955
Epoch 8/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6250 - accuracy: 0.6180
Epoch 9/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6117 - accuracy: 0.6404
Epoch 10/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6011 - accuracy: 0.6629
Epoch 11/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5903 - accuracy: 0.6629
Epoch 12/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5805 - accuracy: 0.6854
Epoch 13/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5700 - accuracy: 0.6966
Epoch 14/18
3/3 [==============================

...

3/3 [==============================] - 0s 2ms/step - loss: 0.7212 - accuracy: 0.5393
Epoch 3/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5843
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6730 - accuracy: 0.6292
Epoch 5/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6536 - accuracy: 0.6404
Epoch 6/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6342 - accuracy: 0.6404
Epoch 7/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6188 - accuracy: 0.7079
Epoch 8/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6046 - accuracy: 0.7416
Epoch 9/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5898 - accuracy: 0.7416
Epoch 10/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5757 - accuracy: 0.7079
Epoch 11/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5666 - accuracy: 0.7640
Epoch 12/12
1/1 [==============================] 

Epoch 13/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5689 - accuracy: 0.7191
Epoch 14/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5561 - accuracy: 0.7303
Epoch 15/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5453 - accuracy: 0.7528
Epoch 16/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7640
Epoch 17/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5214 - accuracy: 0.7865
Epoch 18/18
1/1 [==============================] - 0s 690us/step - loss: 0.5952 - accuracy: 0.7727
Epoch 1/18
3/3 [==============================] - 0s 971us/step - loss: 0.7193 - accuracy: 0.4607
Epoch 2/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6805 - accuracy: 0.5393
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6581 - accuracy: 0.6067
Epoch 4/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6409 - accuracy: 0.6517
Epoch 5/18
3/3 [==============

3/3 [==============================] - 0s 987us/step - loss: 0.4926 - accuracy: 0.8182
Epoch 18/24
3/3 [==============================] - 0s 1ms/step - loss: 0.4837 - accuracy: 0.7955
Epoch 19/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4732 - accuracy: 0.8409
Epoch 20/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4639 - accuracy: 0.8409
Epoch 21/24
3/3 [==============================] - 0s 1ms/step - loss: 0.4524 - accuracy: 0.8636
Epoch 22/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4440 - accuracy: 0.8636
Epoch 23/24
3/3 [==============================] - 0s 998us/step - loss: 0.4342 - accuracy: 0.8864
Epoch 24/24
1/1 [==============================] - 0s 720us/step - loss: 0.5833 - accuracy: 0.6522
Epoch 1/24
3/3 [==============================] - 0s 968us/step - loss: 0.7102 - accuracy: 0.5506
Epoch 2/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6677 - accuracy: 0.5955
Epoch 3/24
3/3 [====================

3/3 [==============================] - 0s 2ms/step - loss: 0.6882 - accuracy: 0.5393
Epoch 4/24
3/3 [==============================] - 0s 1ms/step - loss: 0.6600 - accuracy: 0.5618
Epoch 5/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6400 - accuracy: 0.6067
Epoch 6/24
3/3 [==============================] - 0s 1ms/step - loss: 0.6212 - accuracy: 0.6404
Epoch 7/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6063 - accuracy: 0.6404
Epoch 8/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5901 - accuracy: 0.6742
Epoch 9/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5766 - accuracy: 0.6854
Epoch 10/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5623 - accuracy: 0.7416
Epoch 11/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5500 - accuracy: 0.7640
Epoch 12/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5375 - accuracy: 0.7865
Epoch 13/24
3/3 [==============================]

4/4 [==============================] - 0s 2ms/step - loss: 0.4637 - accuracy: 0.9009
Epoch 23/24
4/4 [==============================] - 0s 2ms/step - loss: 0.4540 - accuracy: 0.8919
Epoch 24/24
4/4 [==============================] - 0s 3ms/step - loss: 0.4469 - accuracy: 0.9009


In [13]:
print(train_hyper_tune(X_train, y_train))

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6953 - accuracy: 0.5341
Epoch 2/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6694 - accuracy: 0.6023
Epoch 3/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6562 - accuracy: 0.6364
Epoch 4/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6446 - accuracy: 0.6477
Epoch 5/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6337 - accuracy: 0.6705
Epoch 6/6
1/1 [==============================] - 0s 898us/step - loss: 0.6900 - accuracy: 0.6087
Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.7428 - accuracy: 0.4382
Epoch 2/6
3/3 [==============================] - 0s 1ms/step - loss: 0.6959 - accuracy: 0.5393
Epoch 3/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6715 - accuracy: 0.5730
Epoch 4/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6518 - accuracy: 0.5730
Epoch 5/6
3/3 [==============================] -

1/1 [==============================] - 0s 954us/step - loss: 0.5711 - accuracy: 0.6522
Epoch 1/12
3/3 [==============================] - 0s 1ms/step - loss: 0.6650 - accuracy: 0.6067
Epoch 2/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.6180
Epoch 3/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6076 - accuracy: 0.6404
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5915 - accuracy: 0.6629
Epoch 5/12
3/3 [==============================] - 0s 5ms/step - loss: 0.5777 - accuracy: 0.6854
Epoch 6/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5662 - accuracy: 0.7303
Epoch 7/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5531 - accuracy: 0.7753
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5445 - accuracy: 0.7753
Epoch 9/12
3/3 [==============================] - 0s 2ms/step - loss: 0.5331 - accuracy: 0.7865
Epoch 10/12
3/3 [==============================] 

1/1 [==============================] - 0s 818us/step - loss: 0.5498 - accuracy: 0.6818
Epoch 1/18
3/3 [==============================] - 0s 983us/step - loss: 0.7106 - accuracy: 0.4545
Epoch 2/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6660 - accuracy: 0.5455
Epoch 3/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6423 - accuracy: 0.5795
Epoch 4/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6234 - accuracy: 0.6136
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6037 - accuracy: 0.6705
Epoch 6/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5896 - accuracy: 0.6932
Epoch 7/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5726 - accuracy: 0.7386
Epoch 8/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5581 - accuracy: 0.7614
Epoch 9/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5436 - accuracy: 0.7727
Epoch 10/18
3/3 [==============================

3/3 [==============================] - 0s 1ms/step - loss: 0.6421 - accuracy: 0.6404
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.6742
Epoch 6/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6106 - accuracy: 0.7416
Epoch 7/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5986 - accuracy: 0.7416
Epoch 8/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5871 - accuracy: 0.7753
Epoch 9/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5765 - accuracy: 0.7865
Epoch 10/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5656 - accuracy: 0.7978
Epoch 11/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5559 - accuracy: 0.8090
Epoch 12/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5457 - accuracy: 0.8090
Epoch 13/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5370 - accuracy: 0.8090
Epoch 14/18
3/3 [==============================

...

1/1 [==============================] - 0s 841us/step - loss: 0.6673 - accuracy: 0.4545
Epoch 1/12
3/3 [==============================] - 0s 986us/step - loss: 0.7289 - accuracy: 0.5506
Epoch 2/12
3/3 [==============================] - 0s 1ms/step - loss: 0.6829 - accuracy: 0.5618
Epoch 3/12
3/3 [==============================] - 0s 1ms/step - loss: 0.6620 - accuracy: 0.5730
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6464 - accuracy: 0.5506
Epoch 5/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6349 - accuracy: 0.5730
Epoch 6/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.6067
Epoch 7/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6151 - accuracy: 0.6067
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.6180
Epoch 9/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5990 - accuracy: 0.6292
Epoch 10/12
3/3 [==============================

3/3 [==============================] - 0s 3ms/step - loss: 0.4696 - accuracy: 0.8090
Epoch 17/18
3/3 [==============================] - 0s 4ms/step - loss: 0.4590 - accuracy: 0.8202
Epoch 18/18
1/1 [==============================] - 0s 894us/step - loss: 0.6455 - accuracy: 0.6818
Epoch 1/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6856 - accuracy: 0.5730
Epoch 2/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6519 - accuracy: 0.5955
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6324 - accuracy: 0.6067
Epoch 4/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6166 - accuracy: 0.6292
Epoch 5/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6039 - accuracy: 0.6517
Epoch 6/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5912 - accuracy: 0.6742
Epoch 7/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5803 - accuracy: 0.6854
Epoch 8/18
3/3 [==============================]

1/1 [==============================] - 0s 791us/step - loss: 0.6814 - accuracy: 0.5909
Epoch 1/24
3/3 [==============================] - 0s 1ms/step - loss: 0.7048 - accuracy: 0.5114
Epoch 2/24
3/3 [==============================] - 0s 1ms/step - loss: 0.6705 - accuracy: 0.5568
Epoch 3/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6489 - accuracy: 0.6136
Epoch 4/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6342 - accuracy: 0.6818
Epoch 5/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6217 - accuracy: 0.6705
Epoch 6/24
3/3 [==============================] - 0s 3ms/step - loss: 0.6102 - accuracy: 0.6932
Epoch 7/24
3/3 [==============================] - 0s 3ms/step - loss: 0.6006 - accuracy: 0.7159
Epoch 8/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5900 - accuracy: 0.7273
Epoch 9/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5799 - accuracy: 0.7500
Epoch 10/24
3/3 [==============================] 

3/3 [==============================] - 0s 1ms/step - loss: 0.5097 - accuracy: 0.7978
Epoch 20/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5012 - accuracy: 0.8202
Epoch 21/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4918 - accuracy: 0.8315
Epoch 22/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4845 - accuracy: 0.8315
Epoch 23/24
3/3 [==============================] - 0s 3ms/step - loss: 0.4747 - accuracy: 0.8427
Epoch 24/24
1/1 [==============================] - 0s 1ms/step - loss: 0.6678 - accuracy: 0.5909
Epoch 1/24
3/3 [==============================] - 0s 1ms/step - loss: 0.7561 - accuracy: 0.5506
Epoch 2/24
3/3 [==============================] - 0s 2ms/step - loss: 0.7036 - accuracy: 0.5730
Epoch 3/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6754 - accuracy: 0.6067
Epoch 4/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6509 - accuracy: 0.6292
Epoch 5/24
3/3 [==============================

4/4 [==============================] - 0s 3ms/step - loss: 0.6486 - accuracy: 0.5676
Epoch 6/24
4/4 [==============================] - 0s 3ms/step - loss: 0.6340 - accuracy: 0.5946
Epoch 7/24
4/4 [==============================] - 0s 2ms/step - loss: 0.6197 - accuracy: 0.6486
Epoch 8/24
4/4 [==============================] - 0s 2ms/step - loss: 0.6059 - accuracy: 0.6937
Epoch 9/24
4/4 [==============================] - 0s 1ms/step - loss: 0.5926 - accuracy: 0.7027
Epoch 10/24
4/4 [==============================] - 0s 3ms/step - loss: 0.5801 - accuracy: 0.7477
Epoch 11/24
4/4 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.7568
Epoch 12/24
4/4 [==============================] - 0s 2ms/step - loss: 0.5545 - accuracy: 0.8018
Epoch 13/24
4/4 [==============================] - 0s 2ms/step - loss: 0.5428 - accuracy: 0.8018
Epoch 14/24
4/4 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.8108
Epoch 15/24
4/4 [=============================

In [14]:
def train_on_entire(X,y,pipe,grid_res):
    # fit pipeline
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    # return the newly trained pipeline
    return pipe

In [15]:
train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test)

0.8950819672131147

In [16]:
train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test)

0.8918032786885246

In [17]:
train_on_entire(X_train,y_train,my_pipe3,my_grid3).score(X_test,y_test)

Epoch 1/18
39/39 [==============================] - 0s 1ms/step - loss: 0.5955 - accuracy: 0.6844
Epoch 2/18
39/39 [==============================] - 0s 1ms/step - loss: 0.4873 - accuracy: 0.7721
Epoch 3/18
39/39 [==============================] - 0s 2ms/step - loss: 0.4120 - accuracy: 0.8270
Epoch 4/18
39/39 [==============================] - 0s 5ms/step - loss: 0.3576 - accuracy: 0.8590
Epoch 5/18
39/39 [==============================] - 0s 3ms/step - loss: 0.3210 - accuracy: 0.8705
Epoch 6/18
39/39 [==============================] - 0s 2ms/step - loss: 0.2957 - accuracy: 0.8762
Epoch 7/18
39/39 [==============================] - 0s 2ms/step - loss: 0.2774 - accuracy: 0.8803
Epoch 8/18
39/39 [==============================] - 0s 3ms/step - loss: 0.2627 - accuracy: 0.8852
Epoch 9/18
39/39 [==============================] - 0s 3ms/step - loss: 0.2515 - accuracy: 0.8951
Epoch 10/18
39/39 [==============================] - 0s 2ms/step - loss: 0.2432 - accuracy: 0.8943
Epoch 11/18
39/39 [

0.8754098415374756